In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/TFT_energy.csv')

In [4]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'time'], infer_datetime_format=True)

In [6]:
df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
0,2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,...,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0,2015-01-01 00:00:00+00:00
1,2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,...,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0,2015-01-01 01:00:00+00:00
2,2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,...,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0,2015-01-01 02:00:00+00:00
3,2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,...,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0,2015-01-01 03:00:00+00:00
4,2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,...,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0,2015-01-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [7]:
df.columns

Index(['time', 'price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump',
       'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(28050, 28)

In [10]:
test_df

,time,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,...,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump,timestamp
28050,2018-03-14 18:00:00+00:00,50.95,37.61,1068.0,5319.0,35044.0,0.0,345.0,102.0,1006.0,...,40.0,1849.0,220.0,1.0,1.0,8.0,14265.0,6.0,761.0,2018-03-14 18:00:00+00:00
28051,2018-03-14 19:00:00+00:00,59.73,48.21,1126.0,5292.0,36230.0,0.0,351.0,101.0,1005.0,...,40.0,1821.0,220.0,4.0,2.0,8.0,13306.0,5.0,149.0,2018-03-14 19:00:00+00:00
28052,2018-03-14 20:00:00+00:00,50.89,41.11,1054.0,4318.0,35065.0,0.0,351.0,103.0,1005.0,...,40.0,1853.0,220.0,1.0,2.0,5.0,12970.0,7.0,168.0,2018-03-14 20:00:00+00:00
28053,2018-03-14 21:00:00+00:00,45.94,36.72,963.0,4047.0,32328.0,0.0,350.0,104.0,1004.0,...,20.0,1853.0,210.0,1.0,1.0,1.0,13090.0,3.0,811.0,2018-03-14 21:00:00+00:00
28054,2018-03-14 22:00:00+00:00,30.59,21.20,853.0,3726.0,29528.0,0.0,329.0,95.0,1004.0,...,0.0,1826.0,220.0,0.0,8.0,5.0,13408.0,6.0,2007.0,2018-03-14 22:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,...,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0,2018-12-31 18:00:00+00:00
35059,2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,...,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0,2018-12-31 19:00:00+00:00
35060,2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,...,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0,2018-12-31 20:00:00+00:00
35061,2018-12-31 21:00:00+00:00,69.89,63.93,2293.0,6562.0,25801.0,0.0,163.0,93.0,1028.0,...,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0,2018-12-31 21:00:00+00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
    freq = 'H'
)

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### training

In [13]:
K = 12
n_hours = 24*3

In [14]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████| 100/100 [02:07<00:00,  1.27s/it, epoch=130/150, avg_epoch_loss=0.646]


100%|████████████████████████████████████████████████| 100/100 [02:10<00:00,  1.31s/it, epoch=150/150, avg_epoch_loss=0.624]


#### predicted y

In [15]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [16]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'price',
    timestamp = 'timestamp',
    feat_dynamic_real = ['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


0
[59.87095642 60.30499649 58.74200439 55.75432205 52.21709061 49.16346359
 44.11409378 41.24279785 39.20806503 38.25613785 38.21086884 38.88300323]
(12,)
------
1
[59.87095642 60.30499649 58.74200439 55.75432205 52.21709061 49.16346359
 44.11409378 41.24279785 39.20806503 38.25613785 38.21086884 38.88300323
 62.59473801 59.63487625 55.65233231 52.08699417 49.10141373 45.45020676
 43.00783157 41.28310776 40.35693359 40.12314606 40.57341385 41.71085739]
(24,)
------
2
[59.87095642 60.30499649 58.74200439 55.75432205 52.21709061 49.16346359
 44.11409378 41.24279785 39.20806503 38.25613785 38.21086884 38.88300323
 62.59473801 59.63487625 55.65233231 52.08699417 49.10141373 45.45020676
 43.00783157 41.28310776 40.35693359 40.12314606 40.57341385 41.71085739
 66.59547424 59.79089355 54.49666214 50.44944763 46.17637253 42.23501205
 39.22305298 37.83225632 37.61227417 38.42789078 40.16893387 43.61796951]
(36,)
------
3
[59.87095642 60.30499649 58.74200439 55.75432205 52.21709061 49.16346359
 

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[59.87095642 60.30499649 58.74200439 55.75432205 52.21709061 49.16346359
 44.11409378 41.24279785 39.20806503 38.25613785 38.21086884 38.88300323
 62.59473801 59.63487625 55.65233231 52.08699417 49.10141373 45.45020676
 43.00783157 41.28310776 40.35693359 40.12314606 40.57341385 41.71085739
 66.59547424 59.79089355 54.49666214 50.44944763 46.17637253 42.23501205
 39.22305298 37.83225632 37.61227417 38.42789078 40.16893387 43.61796951
 62.13158035 56.5907135  52.57625198 48.47396469 44.83169174 41.82791519
 40.22638702 39.76817703 40.42084503 42.23073959 46.06193542 49.98259354
 60.59495163 56.49846268 53.01955032 49.97366333 47.58340454 46.15885544
 45.62090683 46.0667572  47.71570206 50.90443802 53.13196945 53.99287796
 56.98353195 53.31071091 50.67866898 48.57931519 47.17188644 46.65925217
 46.97915649 48.25005722 51.08867264 53.54216003 54.39454269 54.45880127
 52.74949265 50.46240616 48.71887589 47.32577133 46.7887764  47.2170105
 48.40419388 50.76861572 53.38092041 54.2615242  54.

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

...

/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[59.87095642 60.30499649 58.74200439 ... 67.23442841 66.74545288
 66.9464798 ]
(56436,)
------
4703
[59.87095642 60.30499649 58.74200439 ... 67.62404633 67.46677399
 67.75217438]
(56448,)
------
4704
[59.87095642 60.30499649 58.74200439 ... 65.90052795 66.14035797
 67.1930542 ]
(56460,)
------
4705
[59.87095642 60.30499649 58.74200439 ... 71.77074432 72.44467926
 73.27767944]
(56472,)
------
4706
[59.87095642 60.30499649 58.74200439 ... 73.2878418  74.08296204
 74.98069   ]
(56484,)
------
4707
[59.87095642 60.30499649 58.74200439 ... 74.40041351 75.29621124
 75.94530487]
(56496,)
------
4708
[59.87095642 60.30499649 58.74200439 ... 75.83229065 76.3372879
 76.79120636]
(56508,)
------
4709


/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index repr

[59.87095642 60.30499649 58.74200439 ... 76.72109985 77.05358887
 77.30194092]
(56520,)
------
4710
[59.87095642 60.30499649 58.74200439 ... 77.36841583 77.63149261
 77.59585571]
(56532,)
------
4711
[59.87095642 60.30499649 58.74200439 ... 77.07381439 77.01277924
 76.1140976 ]
(56544,)
------
4712
[59.87095642 60.30499649 58.74200439 ... 76.46426392 75.83957672
 74.21222687]
(56556,)
------
4713
[59.87095642 60.30499649 58.74200439 ... 75.11103058 73.23340607
 71.14012909]
(56568,)
------
4714
[59.87095642 60.30499649 58.74200439 ... 71.70517731 69.44138336
 69.38672638]
(56580,)
------
4715
[59.87095642 60.30499649 58.74200439 ... 71.99842072 71.77762604
 74.29177856]
(56592,)
------
4716


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(6930, 12)

In [18]:
test_ds_predict_q50_supervised[0:3]

array([[59.87095642, 60.30499649, 58.74200439, 55.75432205, 52.21709061,
        49.16346359, 44.11409378, 41.24279785, 39.20806503, 38.25613785,
        38.21086884, 38.88300323],
       [62.59473801, 59.63487625, 55.65233231, 52.08699417, 49.10141373,
        45.45020676, 43.00783157, 41.28310776, 40.35693359, 40.12314606,
        40.57341385, 41.71085739],
       [66.59547424, 59.79089355, 54.49666214, 50.44944763, 46.17637253,
        42.23501205, 39.22305298, 37.83225632, 37.61227417, 38.42789078,
        40.16893387, 43.61796951]])

#### actual y

In [19]:
from helper import series_to_supervised, stage_series_to_supervised

In [20]:
test_df_supervised = series_to_supervised(test_df[['price']], n_hours, K)

In [21]:
test_df_supervised.iloc[:, -K:].shape

(6930, 12)

In [22]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [23]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),var1(t+10),var1(t+11)
28122,63.57,73.69,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73
28123,73.69,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66
28124,67.93,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,48.20
28125,64.33,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,48.20,46.77
28126,59.28,55.41,53.16,49.20,48.17,46.76,47.25,46.73,45.66,48.20,46.77,46.58
...,...,...,...,...,...,...,...,...,...,...,...,...
35047,66.64,70.33,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
35048,70.33,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
35049,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30
35050,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30,69.89


#### performance on test set

In [24]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 3.099
RMSE = 4.44
